# Time Series Prediction with RNNs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import numpy as np
import matplotlib.pyplot as plt

## Generate Time Series Data

In [ ]:
# Generate synthetic time series data
def generate_time_series(length=1000, freq=0.1):
    t = np.linspace(0, 10, length)
    signal = np.sin(2 * np.pi * freq * t) + 0.1 * np.random.randn(length)
    return signal

signal = generate_time_series(length=1000)

plt.figure(figsize=(12, 3))
plt.plot(signal)
plt.title('Time Series Signal')
plt.xlabel('Time')
plt.ylabel('Value')
plt.grid(True)
plt.show()

## Prepare Sequences

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, signal, seq_length=20):
        self.signal = torch.from_numpy(signal).float()
        self.seq_length = seq_length
    
    def __len__(self):
        return len(self.signal) - self.seq_length
    
    def __getitem__(self, idx):
        x = self.signal[idx:idx + self.seq_length].unsqueeze(1)  # (seq_len, 1)
        y = self.signal[idx + self.seq_length]  # scalar
        return x, y

seq_length = 20
dataset = TimeSeriesDataset(signal, seq_length=seq_length)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

## LSTM Model

In [ ]:
class LSTMTimeSeriesNet(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=2, output_size=1):
        super(LSTMTimeSeriesNet, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # x shape: (batch, seq_len, 1)
        lstm_out, (h_n, c_n) = self.lstm(x)
        # Use the last output
        last_output = lstm_out[:, -1, :]
        output = self.fc(last_output)
        return output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMTimeSeriesNet(input_size=1, hidden_size=32, num_layers=2).to(device)
print(model)

## Train the Model

In [ ]:
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 20
losses = []

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        
        outputs = model(x).squeeze()
        loss = loss_fn(outputs, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    epoch_loss /= len(train_loader)
    losses.append(epoch_loss)
    
    if (epoch + 1) % 5 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}')

## Make Predictions

In [ ]:
model.eval()
predictions = []
actual = []

with torch.no_grad():
    for x, y in DataLoader(dataset, batch_size=1):
        x = x.to(device)
        pred = model(x).squeeze().cpu().numpy()
        predictions.append(pred)
        actual.append(y.numpy())

predictions = np.array(predictions)
actual = np.array(actual).flatten()

# Plot results
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

axes[0].plot(losses)
axes[0].set_title('Training Loss')
axes[0].set_ylabel('Loss')
axes[0].set_xlabel('Epoch')
axes[0].grid(True)

axes[1].plot(actual, label='Actual', alpha=0.7)
axes[1].plot(predictions, label='Predicted', alpha=0.7)
axes[1].set_title('Time Series Predictions')
axes[1].set_ylabel('Value')
axes[1].set_xlabel('Time Step')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

mse = np.mean((actual - predictions) ** 2)
print(f'\nMean Squared Error: {mse:.4f}')